<a href="https://colab.research.google.com/github/whoiswentz/tcc-neural-network/blob/master/densenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.densenet import DenseNet169
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Flatten, Dense
from keras.models import Model

from keras import losses
from keras import optimizers
from keras import activations

import tensorflow as tf

import numpy as np

In [0]:
# Mount the Google Drive folder to get the dataset
from google.colab import drive
drive.mount(GOOGLE_DRIVE_MOUNT_DIR)

In [0]:
GOOGLE_DRIVE_MOUNT_DIR = "/content/drive"

TRAIN_PATH = "../content/drive/My Drive/BreakHis/Dataset_Resized/Binary/train"
TEST_PATH = "../content/drive/My Drive/BreakHis/Dataset_Resized/Binary/test"

MULTICLASS_TRAIN_PATH = "../content/drive/My Drive/BreakHis/Dataset_Resized/8Class/train"
MULTICLASS_TEST_PATH = "../content/drive/My Drive/BreakHis/Dataset_Resized/8Class/test"

CHECK_POINT_PATH = "../content/drive/My Drive/BreakHis/{}.ckpt"

BATCH_SIZE = 128
EPOCHS = 50

In [0]:
def create_densenet_model(output_space):
    # Get back the convolutional part of a Densenet network trained on ImageNet
    base_densenet_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(175, 115, 3))

    #Add a layer where input is the output of the second last layer 
    x = Flatten(name="flatten")(base_densenet_model.layers[-2].output)
    x = Dense(output_space, activation=activations.softmax, name='predictions')(x)

    #Create your own model
    densenet = Model(input=base_densenet_model.get_input_at(0), output=x)
    
    # Freezing the first five layers
    half_number_of_layers = int(np.floor(len(densenet.layers) / 2))
    for layer in densenet.layers[:half_number_of_layers]:
        layer.trainable = False

    # Compoling the Densenet model
    densenet.compile(loss="categorical_crossentropy", optimizer="rmsprop")
    
    # Show the neural network summary
    # densenet.summary()

    return densenet

# Tumor Class

The tumos class has two categories, benign and milignants.

In [0]:
densenet_tumor_class = create_densenet_model(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
# TUMOR CLASS

datagen = ImageDataGenerator()

train_data_generator = datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(350, 230),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

test_data_generator = datagen.flow_from_directory(
    TEST_PATH,
    target_size=(350, 230),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

train_data_generator.class_indices
test_data_generator.class_indices

# Create checkpoint callback
# To resume the training

tumor_class_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    CHECK_POINT_PATH.format("tumor_class"), 
    save_weights_only=False, 
    period=5,
    verbose=1)

In [0]:
steps_per_epoch = int(
    np.ceil(densenet_tumor_class.input_shape[1] / float(BATCH_SIZE)))

densenet_tumor_class.fit_generator(
    generator=train_data_generator,
    validation_data=test_data_generator,
    validation_steps=steps_per_epoch,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    callbacks = [tumor_class_cp_callback],
    workers=4)

# Tumor Type

In [0]:
densenet_tumor_type = create_densenet_model(10)

In [0]:
# TUMOR TYPE

datagen = ImageDataGenerator()

tumor_type_train_data_generator = datagen.flow_from_directory(
    MULTICLASS_TRAIN_PATH,
    target_size=(350, 230),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

tumor_type_test_data_generator = datagen.flow_from_directory(
    MULTICLASS_TEST_PATH,
    target_size=(350, 230),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

tumor_type_train_data_generator.class_indices
tumor_type_test_data_generator.class_indices

# Create checkpoint callback
# To resume the training

tumor_type_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    CHECK_POINT_PATH.format("tumor_type"), 
    save_weights_only=False, 
    period=5,
    verbose=1)

In [0]:
steps_per_epoch = int(
    np.ceil(densenet_tumor_class.input_shape[1] / float(BATCH_SIZE)))

densenet_tumor_class.fit_generator(
    generator=train_data_generator,
    validation_data=test_data_generator,
    validation_steps=steps_per_epoch,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    callbacks = [tumor_type_cp_callback],
    workers=4)

In [0]:
import pathlib

# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
print(checkpoints)

print("TUMOR CLASS ACC")
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))